# Initialize Environment

In [ ]:
import tensorflow as tf
import skopt
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
import numpy as np
import datetime
import random
import os
from shutil import rmtree

In [ ]:
%matplotlib inline
%load_ext tensorboard.notebook

In [ ]:
LOGS_PATH = os.path.join('.', 'logs', 'fit', 'mnist')
MODEL_FILE_NAME = 'tf_mnist_dnn.model'

# Load Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
sample_pos = random.randint(1, x_train.shape[0])
print(f"Image at {sample_pos}")
plt.imshow(x_train[sample_pos - 1], cmap = plt.cm.binary)

# Normalize Data

In [ ]:
x_train = x_train.astype(float)/255
x_test = x_test.astype(float)/255

# Define Model

In [ ]:
def create_model(learning_rate, num_dense_layers, num_dense_nodes, activation):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    for i in range(num_dense_layers - 1):
        model.add(tf.keras.layers.Dense(num_dense_nodes, activation=activation))
    if num_dense_layers > 1:
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(10, activation = tf.nn.softmax))
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

# Train & Verify Model

In [ ]:
def log_dir_path(learning_rate, num_dense_layers, num_dense_nodes, activation):
    time_stamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    log_dir_name_pattern = "lr{0:.0e}_l{1}_n{2}_a{3}_{4}"
    log_dir_name = log_dir_name_pattern.format(learning_rate,
                                               num_dense_layers,
                                               num_dense_nodes,
                                               activation,
                                               time_stamp)
    log_dir = os.path.join(LOGS_PATH, log_dir_name)

    return log_dir
    

In [ ]:
def fit_model(learning_rate, num_dense_layers, num_dense_nodes, activation):
    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation)
    
    log_dir = log_dir_path(learning_rate, num_dense_layers,
                           num_dense_nodes, activation)
    
    tensor_board_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                           histogram_freq=1)
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                               patience=2,
                                                               restore_best_weights=True)
   
    history = model.fit(x=x_train,
                        y=y_train,
                        epochs = 20,
                        validation_data=(x_test, y_test), 
                        callbacks=[tensor_board_callback, early_stopping_callback])
    accuracy = history.history['val_accuracy'][early_stopping_callback.stopped_epoch]
    
    return model, accuracy

In [ ]:
# Clear any logs from previous runs
if os.path.isdir(LOGS_PATH):
    rmtree(LOGS_PATH)

In [ ]:
initial_parameters = [1e-3, 2, 128, 'relu']
model, accuracy = fit_model(*initial_parameters)
print()
print("Accuracy: {0:.2%}".format(accuracy))
del model
tf.keras.backend.clear_session()

# Hyperparameter Optimiziation
Using Bayesian Optimiziation

In [ ]:
dim_learning_rate = skopt.space.Real(low=1e-6, high=1e-1, prior='log-uniform', name='learning_rate')
dim_num_dense_layers = skopt.space.Integer(low=1, high=5, name='num_dense_layers')
dim_num_dense_nodes = skopt.space.Integer(low=5, high=512, name='num_dense_nodes')
dim_activation = skopt.space.Categorical(categories=['relu', 'sigmoid'], name='activation')
dimensions = [dim_learning_rate, dim_num_dense_layers, dim_num_dense_nodes, dim_activation]

In [ ]:
best_run = {
    'accuracy': 0.0,
    'learning_rate': None,
    'num_dense_layers': None,
    'num_dense_nodes': None,
    'activation': None
}
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_dense_nodes, activation):
    
    print('Learning rate: {0:.1e}'.format(learning_rate))
    print('Number of dense layers:', num_dense_layers)
    print('Number of dense nodes:', num_dense_nodes)
    print('Activation function:', activation)
    print()
    
    model, accuracy = fit_model(learning_rate=learning_rate,
                                num_dense_layers=num_dense_layers,
                                num_dense_nodes=num_dense_nodes,
                                activation=activation)
    
    global best_run
    
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print("Best Accuracy so far: {0:.2%}".format(best_run['accuracy']))
    print()
    
    if accuracy > best_run['accuracy']:
        model.save(MODEL_FILE_NAME)
        best_run['accuracy'] = accuracy
        best_run['learning_rate'] = learning_rate
        best_run['num_dense_layers'] = num_dense_layers
        best_run['num_dense_nodes'] = num_dense_nodes
        best_run['activation'] = activation
    
    del model
    tf.keras.backend.clear_session()
    
    # Scikit-optimize tries to find a set of hyper-parameters with the LOWEST fitness-value
    return -accuracy

In [ ]:
search_result = skopt.gp_minimize(func=fitness,
                                  dimensions=dimensions,
                                  acq_func='EI',
                                  n_calls=40,
                                  x0=initial_parameters)

In [ ]:
print("Best Accuracy: {0:.2%}".format(best_run['accuracy']))
print('Best set of hyperparameters:')
print('  Learning rate: {0:.1e}'.format(best_run['learning_rate']))
print('  Number of dense layers:', best_run['num_dense_layers'])
print('  Number of dense nodes:', best_run['num_dense_nodes'])
print('  Activation function:', best_run['activation'])

In [ ]:
%tensorboard --logdir './logs/fit/mnist'

# Use Model

In [ ]:
!test_mnist